# [Module 1.2] 로컬 모드 및 스크립트 모드로 훈련 (SageMaker 사용)

### 본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 기본 환경 세팅
- 데이터 세트를 S3에 업로드
- 스크립트 모드의 코드 작성 방법
- 훈련 코드 확인
- 로컬 모드로 훈련 실행
- SageMaker Host Mode 로 훈련

---

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%store -r local_model_dir

In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-cnn-cifar10"

role = sagemaker.get_execution_role()

#### 로컬의 GPU, CPU 여부로 instance_type 결정

In [4]:
import os
import subprocess


try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
    else:
        instance_type = "local"        
except:
    pass

print("Instance type = " + instance_type)

Instance type = local_gpu


### 데이터 세트를 S3에 업로드


In [5]:
inputs = sagemaker_session.upload_data(path="../data", bucket=bucket, key_prefix="data/cifar10")
print("s3 inputs: ", inputs)

s3 inputs:  s3://sagemaker-us-east-1-057716757052/data/cifar10


In [6]:
! aws s3 ls {inputs} --recursive

2021-09-27 14:12:16        158 data/cifar10/cifar-10-batches-py/batches.meta
2021-09-27 14:12:12   31035704 data/cifar10/cifar-10-batches-py/data_batch_1
2021-09-27 14:12:10   31035320 data/cifar10/cifar-10-batches-py/data_batch_2
2021-09-27 14:12:12   31035999 data/cifar10/cifar-10-batches-py/data_batch_3
2021-09-27 14:12:11   31035696 data/cifar10/cifar-10-batches-py/data_batch_4
2021-09-27 14:12:11   31035623 data/cifar10/cifar-10-batches-py/data_batch_5
2021-09-27 14:12:11         88 data/cifar10/cifar-10-batches-py/readme.html
2021-09-27 14:12:10   31035526 data/cifar10/cifar-10-batches-py/test_batch
2021-09-27 14:12:09  170498071 data/cifar10/cifar-10-python.tar.gz


## 스크립트 모드의 코드 작성 방법
- ![script_mode_example.png](img/script_mode_example.png)

## 훈련 코드 확인
- 아래의 코드는 전형적인 스크립트 모드의 코드 작성 방법을 따르고 있습니다.
- 훈련 함수는 `from train_lib import train` 로서 이전 노트북의 **[세이지 메이커 없이]** 작성한 스크래치 버전에서 사용한 훈련 함수와 동일 합니다.


In [7]:
train_code = 'source/train.py'
!pygmentize {train_code}

import argparse
import os
import json

from train_lib import train


if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    ##################################
    #### 사용자 정의 커맨드 인자
    ##################################
    
    parser.add_argument(
        "--workers",
        type=int,
        default=2,
        metavar="W",
        help="number of data loading workers (default: 2)",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=1,
        metavar="E",
        help="number of total epochs to run (default: 2)",
    )
    parser.add_argument(
        "--batch_size", type=int, default=4, metavar="BS", help="batch size (default: 4)"
    )
    parser.add_argument(
        "--lr",
        type=float,
        default=0.001,
        metavar="LR",
        help="initial learning rate (default: 0.001)",
    )
    parser.add_argument(
        "--momentum", type=float, default=0.9, metavar="M", help="momentum (default: 0.9)"
    )
    parser

### 로컬에 있는 데이타 세트의 위치를 지정 합니다.

In [8]:
local_inputs = "file://../data"

### 로컬 모드로 훈련 실행
- 아래의 두 라인이 로컬모드로 훈련을 지시 합니다.
```python
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
```

In [9]:
from sagemaker.pytorch import PyTorch
import os
import subprocess


local_cifar10_estimator = PyTorch(
    entry_point="train.py",    
    source_dir='source',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
    hyperparameters={'epochs': 1, 
                     'lr': 0.1,
                     'batch_size': 16
                    }                      
    
)
local_cifar10_estimator.fit(local_inputs)

Creating rz56mlhkp9-algo-1-yx9pc ... 
Creating rz56mlhkp9-algo-1-yx9pc ... done
Attaching to rz56mlhkp9-algo-1-yx9pc
rz56mlhkp9-algo-1-yx9pc | 2021-09-27 14:12:49,852 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
rz56mlhkp9-algo-1-yx9pc | 2021-09-27 14:12:49,932 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
rz56mlhkp9-algo-1-yx9pc | 2021-09-27 14:12:49,935 sagemaker_pytorch_container.training INFO     Invoking user training script.
rz56mlhkp9-algo-1-yx9pc | 2021-09-27 14:12:50,134 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
rz56mlhkp9-algo-1-yx9pc | /opt/conda/bin/python3.6 -m pip install -r requirements.txt
rz56mlhkp9-algo-1-yx9pc | Requirement already satisfied: torch==1.8.1 in /opt/conda/lib/python3.6/site-packages (from -r requirements.txt (line 1)) (1.8.1)
rz56mlhkp9-algo-1-yx9pc | Requirement already satisfied: torchvision==0.9.1 in /opt/conda/lib/python3

## SageMaker Host Mode 로 훈련
- `cifar10_estimator.fit(inputs, wait=False)`
    - 입력 데이터를 inputs로서 S3 의 경로를 제공합니다.
    - wait=False 로 지정해서 async 모드로 훈련을 실행합니다. 
        - 실행 경과는 아래의 cifar10_estimator.logs() 에서 확인 합니다.

In [10]:
print("inputs: \n", inputs)

inputs: 
 s3://sagemaker-us-east-1-057716757052/data/cifar10


In [11]:
from sagemaker.pytorch import PyTorch

instance_type = 'ml.p3.2xlarge'

cifar10_estimator = PyTorch(
    entry_point="train.py",    
    source_dir='source',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type,
    session = sagemaker.Session(), # 세이지 메이커 세션
    hyperparameters={'epochs': 10, 
                     'lr': 0.001,
                     'batch_size': 16                     
                    }                      
    
)
cifar10_estimator.fit(inputs, wait=False)

In [12]:
cifar10_estimator.logs()

2021-09-27 14:14:28 Starting - Starting the training job...
2021-09-27 14:14:57 Starting - Launching requested ML instancesProfilerReport-1632752068: InProgress
......
2021-09-27 14:15:57 Starting - Preparing the instances for training.........
2021-09-27 14:17:17 Downloading - Downloading input data...
2021-09-27 14:17:57 Training - Downloading the training image.......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-09-27 14:21:38,845 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-09-27 14:21:38,869 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-09-27 14:21:45,098 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-09-27 14:21:45,513 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt

20

## 모델 아티펙트 저장
- S3 에 저장된 모델 아티펙트를 저장하여 추론시 사용합니다.

In [14]:
artifact_path = cifar10_estimator.model_data
print("artifact_path: ", artifact_path)

%store artifact_path

artifact_path:  s3://sagemaker-us-east-1-057716757052/pytorch-training-2021-09-27-14-14-28-250/output/model.tar.gz
Stored 'artifact_path' (str)
